In [5]:
import json
import re
import subprocess

from multiprocessing import Process, JoinableQueue as Queue
from os import listdir, path
from queue import Empty as QueueEmptyException

VIDEO_PATH = "/home/vmubuntu/OitoDeJaneiro/vids/FULL-0801"
CAMERA_DB_PATH = "./metadata/cameras.json"

In [6]:
with open(CAMERA_DB_PATH, "r") as f:
  camera_data = json.load(f)

In [ ]:
for io_dir in camera_data.keys():
  input_dir_path = path.join(VIDEO_PATH, io_dir)
  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])

  m_cam_res = camera_data[io_dir]["resolution"]
  cam_width = m_cam_res[0] if 0 < len(m_cam_res) else 0
  cam_height = m_cam_res[1] if 1 < len(m_cam_res) else 0

  for io_file in input_files:
    input_file_path = path.join(input_dir_path, io_file)
    vid = cv2.VideoCapture(input_file_path)

    m_width = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
    m_height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)

    if cam_width == 0 or cam_height == 0:
      cam_width, cam_height = m_width, m_height
    elif abs(cam_width - m_width) > 2:
      print("ERROR: video resolution", io_file, cam_width, m_width)
    elif abs(cam_height - m_height) > 2:
      print("ERROR: video resolution", io_file, cam_height, m_height)
    else:
      cam_width, cam_height = m_width, m_height
  
  camera_data[io_dir]["resolution"] = [cam_width, cam_height]

In [ ]:
with open(CAMERA_DB_PATH, "w") as f:
  json.dump(camera_data, f, indent=2)